**TODO:**

* currently only using max iter to halt
* function to select subset of image based on PSF size and l,m position

### Högbom's Method

Psuedo-code:

$\textbf{input: } I^{D}(l,m), \ \textrm{PSF}(l,m), \ \gamma, \ f_{\textrm{thresh}}, \ N$

$\textbf{initialize: } S^{\textrm{model}} \leftarrow \{\}, I^{\textrm{res}} \leftarrow I^{D}, i \leftarrow 0$

$\textbf{while} \ \textrm{any}(I^{\textrm{res}} > f_{\textrm{thresh}}) \ \textrm{or} \ i \leq N \ \textbf{do:}$

$\qquad l_{\textrm{max}}, m_{\textrm{max}} \leftarrow \underset{l,m}{\operatorname{argmax}} I^{\textrm{res}}(l,m)$

$\qquad f_{\textrm{max}} \leftarrow I^{D}(l_{\textrm{max}}, m_{\textrm{max}})$

$\qquad I^{\textrm{res}} \leftarrow I^{\textrm{res}} - \gamma \cdot f_{\textrm{max}} \cdot \textrm{PSF}(l+l_{\textrm{max}}, m+m_{\textrm{max}})$

$\qquad S^{\textrm{model}} \leftarrow S^{\textrm{model}} + \{l_{\textrm{max}}, m_{\textrm{max}}: \gamma \cdot f_{\textrm{max}}\}$

$\textbf{ouput: } S^{\textrm{model}}, I^{\textrm{res}}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.io import fits
import aplpy

#Disable astropy/aplpy logging
import logging
logger0 = logging.getLogger('astropy')
logger0.setLevel(logging.CRITICAL)
logger1 = logging.getLogger('aplpy')
logger1.setLevel(logging.CRITICAL)

In [ ]:
#input parameters
gain = 0.1 #loop gain, range: 0 < gain < 1
niter = 10 #number of loop iterations
fthresh = 0. #minimum flux threshold to deconvolve

In [ ]:
#input images: dirty, PSF
#assuming unpolarized, single frequency image
fh = fits.open('../data/fits/deconv/KAT-7_6h60s_dec-30_10MHz_10chans_uniform_n100-dirty.fits')
dirtyImg = fh[0].data[0,0]
fh = fits.open('../data/fits/deconv/KAT-7_6h60s_dec-30_10MHz_10chans_uniform_n100-psf.fits')
#zero-pad the PSF
psfImg = fh[0].data[0,0]
padPsfImg = np.pad(psfImg, ((int(psfImg.shape[0]/2.),
                             int(psfImg.shape[0]/2.)),
                            (int(psfImg.shape[1]/2.),
                             int(psfImg.shape[1]/2.))), mode='constant', constant_values=(0,0))

In [ ]:
#Initalization
skyModel = [] #initialize empty model
residImg = np.copy(dirtyImg) #copy the dirty image to the initial residual image
i = 0 #number of iterations
midPSFl = int(padPsfImg.shape[0]/2)
midPSFm = int(padPsfImg.shape[1]/2)

In [ ]:
#while np.max(residImg) > fthresh or i < niter:
while i < niter:
    lmax, mmax = np.unravel_index(residImg.argmax(), residImg.shape) #get pixel position of maximum value
    fmax = residImg[lmax, mmax] #flux value of maximum pixel
    #residImg -= gain * fmax * padPsfImg[]
    i +=1